In [ ]:
# | default_exp ds.tplot.trans
# | export
from pydantic import (
    BaseModel,
    ConfigDict,
    model_validator,
)

import pyspedas
from pytplot import options, data_quants, split_vec, join_vec
from pyspedas.cotrans.minvar_matrix_make import minvar_matrix_make
from pyspedas import tvector_rotate
from pyspedas.analysis.tvectot import tvectot

from typing import Callable

In [ ]:
# | export
# if "slice-1" in config.trans:
#     tvar = split_vec(tvar)[:1]
# if "slice-3" in config.trans:
#     tvar = join_vec(split_vec(tvar)[:3])
# if "mva" in config.trans:
#     minvar_matrix_make(tvar)
#     tvar = tvector_rotate(tvar + "_mva_mat", tvar)[0]
#     legend_names = [r"$B_l$", r"$B_m$", r"$B_n$"]

# if "magnitude" in trans:
#     tvar2plot = tvectot(tvar, join_component=False)
#     options(tvar2plot, "legend_names", None)

# if "magnitude_join" in trans:
#     tvar2plot = tvectot(tvar, join_component=True)
#     legend_names = legend_names + [r"$B_{total}$"]
#     options(tvar2plot, "legend_names", legend_names)


def magnitude(tvar: str):
    tvar2plot = tvectot(tvar, join_component=False)
    options(tvar2plot, "legend_names", None)
    return tvar2plot


def mva(tvar: str, prefix="B"):
    minvar_matrix_make(tvar)
    tvar2plot = tvector_rotate(tvar + "_mva_mat", tvar)[0]
    options(
        tvar2plot,
        "legend_names",
        [rf"${prefix}_l$", rf"${prefix}_m$", rf"${prefix}_n$"],
    )
    return tvar2plot


def magnitude_join(tvar: str):
    tvar2plot = tvectot(tvar, join_component=True)

    data_quants[tvar2plot].attrs["plot_options"]["yaxis_opt"]["legend_names"].append(
        r"$B_{total}$"
    )
    return tvar2plot


In [ ]:
#| export
class TransformOp(BaseModel):
    """Transformation operation"""

    # allow addtional fields
    model_config = ConfigDict(extra="allow")

    name: str

    transform_func: Callable[[str], str] = None

    @model_validator(mode="after")
    def map_transform_func(self):
        transform_func_maps = {
            "avg": pyspedas.avg_data,
            "slice-1": lambda tvar: split_vec(tvar)[:1],
            "slice-3": lambda tvar: join_vec(split_vec(tvar)[:3]),
            "mva": mva,
            "magnitude": magnitude,
            "magnitude_join": magnitude_join,
        }
        self.transform_func = transform_func_maps.get(self.name)
        return self